In [ ]:
!pip install pydub
!pip install transformers
!pip install torch

from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
import torch
import os

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
#model_name = "bofenghuang/whisper-large-v2-cv11-german"  # or any other model you want to use

# Initialize the Whisper model pipeline for automatic speech recognition
pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    chunk_length_s=30,
    device=device,
)

# Load model and processor for multilingual support
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="German", task="transcribe")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [ ]:
# Google Drive benutzen um Dateien permanent zu speichern

from google.colab import drive
drive.mount('/mnt/drive')


Mounted at /mnt/drive


# Durchführen der Transkription für jede Gruppe

Nach der Initialisierung der drive-direcorys muss auch die nächste Zelle mit ausgeführt werden!

In [ ]:
drive_directory = '/mnt/drive/MyDrive/MasterArbeitDaten/LokaleGruppen/schwurbelarchiv-DIxrNAFZYl/audio'
output_directory = '/mnt/drive/MyDrive/MasterArbeitDaten/LokaleGruppen/schwurbelarchiv-DIxrNAFZYl/audio/transcript'
if not os.path.exists(output_directory):
        os.makedirs(output_directory)

In [ ]:
import os
import pandas as pd
from pydub import AudioSegment

def get_mp3_files_info(drive_directory):
    # List all files in the given directory
    files = os.listdir(drive_directory)

    # Filter the list to include only MP3 files
    mp3_files = [file for file in files if file.endswith('.mp3')]

    # List for storing file information
    file_info = []

    # Iterate through each MP3 file to extract information
    for file in mp3_files:
        file_path = os.path.join(drive_directory, file)

        # Use PyDub to load the audio file
        audio = AudioSegment.from_file(file_path)

        # Get file size
        file_size = os.path.getsize(file_path)

        # Get duration and sample rate
        duration = len(audio) / 1000  # Duration in seconds
        sampling_rate = audio.frame_rate

        # Store the information for each file
        file_info.append({
            'file_name': file,
            'file_path': file_path,
            'duration': duration,
            'sampling_rate': sampling_rate,
            'file_size': file_size
        })

    # Create a DataFrame from the list of file information
    df = pd.DataFrame(file_info)
    return df


# Create the DataFrame
df = get_mp3_files_info(drive_directory)



In [ ]:
print(df)

                                             file_name  \
0    Rescue-Lauren_Daigle_video_2020-07-30_13-07-27...   
1                   video_2533@27-12-2020_11-08-27.mp3   
2                                          passout.mp3   
3                   video_2521@26-12-2020_22-00-50.mp3   
4            Jen Murray - Jen's Flat Earth Academy.mp3   
..                                                 ...   
348                            VID-20201219-WA0018.mp3   
349                          2_5402413338984450605.mp3   
350                          2_5400017915464387157.mp3   
351                    file_24@28-12-2020_16-15-21.mp3   
352                                          video.mp3   

                                             file_path  duration  \
0    /mnt/drive/MyDrive/MasterArbeitDaten/LokaleGru...    140.02   
1    /mnt/drive/MyDrive/MasterArbeitDaten/LokaleGru...     74.36   
2    /mnt/drive/MyDrive/MasterArbeitDaten/LokaleGru...     69.57   
3    /mnt/drive/MyDrive/MasterA

In [ ]:
import librosa

def load_and_resample_audio(file_path, target_sample_rate=16000):
    try:
        audio_path = df["file_path"]
        waveform, original_sample_rate = librosa.load(file_path, sr=None, mono=True)
        waveform_resampled = librosa.resample(waveform, orig_sr=original_sample_rate, target_sr=target_sample_rate)
        return waveform_resampled
    except Exception as e:
        print(f"Error loading file {file_path}: {e}")
        return None


In [ ]:
def transcribe_waveform(waveform_resampled, language='german'):
    try:
        forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task="transcribe")
        transcription = ""

        for i in range(0, len(waveform_resampled), 16000 * 30):  # 30 seconds chunks
            chunk = waveform_resampled[i:i + 16000 * 30]
            input_features = processor(chunk, sampling_rate=16000, return_tensors="pt").input_features
            predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids) #, forced_decoder_ids=forced_decoder_ids)
            chunk_transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
            transcription += " " + chunk_transcription
        return transcription.strip()
    except Exception as e:
        print(f"Error during transcription: {e}")
        return ""


Nächster Schritt: skip transcription if txt file already exists

In [ ]:
import os

def apply_transcription_to_dataframe(df, output_directory):
    filtered_index = df['sampling_rate'] > 0

    #Calculate the total number of transcripts to process
    total_transcripts = df[filtered_index].shape[0]
    processed_count = 0

    for index, row in df[filtered_index].iterrows():
        # Construct the transcript file name based on the audio file name
        transcript_filename = os.path.splitext(row['file_name'])[0] + '_transcript.txt'
        # Construct the full path for the transcript file
        transcript_file_path = os.path.join(output_directory, transcript_filename)

        # Check if the transcript file already exists
        if os.path.exists(transcript_file_path):
            # If the file exists, print a message and skip to the next row
            print(f"Transcript file already exists for {row['file_name']}, skipping transcription.")
            continue

        # If the transcript file does not exist, proceed with processing
        print(f"Processing file: {row['file_name']} at path: {row['file_path']}")
        waveform_resampled = load_and_resample_audio(row['file_path'])
        if waveform_resampled is not None:
            transcript = transcribe_waveform(waveform_resampled, language=row.get('language', 'german'))

            # Save the transcript to a file
            with open(transcript_file_path, 'w', encoding='utf-8') as file:
                file.write(transcript)
            print(f"Saved transcript for {row['file_name']} to {transcript_file_path}")
        else:
            print(f"Failed to load or resample audio for {row['file_name']}")
        processed_count += 1

        # Calculate and print the number of transcripts left
        transcripts_left = total_transcripts - processed_count
        print(f"{transcripts_left} transcripts left to process.")


In [ ]:
# Now use this file path in your function call
apply_transcription_to_dataframe(df, output_directory)
